In [2]:
#Load the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

for text preprocessing we need :
<br>- Lowercase letters.
<br>- Removing HTML tags.
<br>- Removing URLs.
<br>- Removing punctuation.
<br>- Chat Words Treatment.
<br>- Spelling Correction.
<br>- Removing stop words
<br>- Handling Emojies
<br>- Tokenization
<br>- Stemming
<br>- Lemmatization

##Importing data from kaggle to colab



In [3]:
# Install Kaggle package
!pip install kaggle

# Upload kaggle.json
from google.colab import files
files.upload()

# Set up Kaggle API credentials
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

# Unzip the dataset
!unzip imdb-dataset-of-50k-movie-reviews.zip


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 43% 11.0M/25.7M [00:00<00:00, 115MB/s]
100% 25.7M/25.7M [00:00<00:00, 180MB/s]
Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [4]:
# Load the dataset into a pandas DataFrame
import pandas as pd

df = pd.read_csv('IMDB Dataset.csv')
print(df.head())


                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [5]:
set(df['sentiment'])

{'negative', 'positive'}

In [6]:
# To tokanize Output column
df['sentiment'] = df['sentiment'].replace({'negative': 0, 'positive': 1})

In [7]:
print(df.head())

                                              review  sentiment
0  One of the other reviewers has mentioned that ...          1
1  A wonderful little production. <br /><br />The...          1
2  I thought this was a wonderful way to spend ti...          1
3  Basically there's a family where a little boy ...          0
4  Petter Mattei's "Love in the Time of Money" is...          1


##1. Lowercase letters.

In [8]:
df['review'] = df['review'].str.lower()
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production. <br /><br />the...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically there's a family where a little boy ...,0
4,"petter mattei's ""love in the time of money"" is...",1


## 2. Remove HTML Tags

Method 1 by using re

In [9]:
import re

def remove_html_tags(text):
  pattern = re.compile('<.*?>')
  return pattern.sub(r"",text)

In [10]:
# E.g.
text ="<html><body><p> Movie 1</p><p> Actor - Salman Khan</p><p> Click here to <a href='http://google.com'>know more</a></p></body></html>"
remove_html_tags(text)

' Movie 1 Actor - Salman Khan Click here to know more'

In [11]:
# apply to our dataset
df['review'] = df['review'].apply(remove_html_tags)
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production. the filming tec...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically there's a family where a little boy ...,0
4,"petter mattei's ""love in the time of money"" is...",1


Method 2 using BeautifulSoup

In [12]:
from bs4 import BeautifulSoup

In [13]:
def remove_html_tags(text):
    return BeautifulSoup(text, 'html.parser').get_text()


In [14]:
# E.g.
text ="<html><body><p> Movie 1</p><p> Actor - Salman Khan</p><p> Click here to <a href='http://google.com'>know more</a></p></body></html>"
print(remove_html_tags(text))

 Movie 1 Actor - Salman Khan Click here to know more


In [15]:
df['review'] = df['review'].apply(remove_html_tags)
df.head()

<ipython-input-13-6d6e64af61d2>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(text, 'html.parser').get_text()


,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production. the filming tec...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically there's a family where a little boy ...,0
4,"petter mattei's ""love in the time of money"" is...",1


## 3. Remove URLs

In [16]:
def remove_urls(text):
  return re.sub('http\S+|www\S+|https\S+', '', text, flags= re.MULTILINE)

In [17]:
# Eg.
text1 = 'Check out my notebook https://www.kaggle.com/campusx/notebook8223fc1abb'
remove_urls(text1)

'Check out my notebook '

In [18]:
df['review'] = df['review'].apply(remove_urls)
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production. the filming tec...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically there's a family where a little boy ...,0
4,"petter mattei's ""love in the time of money"" is...",1


## 4. Remove Punctuations

In [19]:
import string
punc = string.punctuation
print(punc)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


* The code defines a function, remove_punctuations, that takes a text input and removes all punctuation characters from it using
* The translate method with a translation table created by str.maketrans. This function effectively cleanses the text of punctuation symbols.

In [20]:
def remove_punctuations(text):
  return text.translate(str.maketrans('','', punc))

In [21]:
text = '"Yes, I did call her", Maggie said.'

In [22]:
remove_punctuations(text)

'Yes I did call her Maggie said'

In [23]:
df['review'] = df['review'].apply(remove_punctuations)
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production the filming tech...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically theres a family where a little boy j...,0
4,petter matteis love in the time of money is a ...,1


## 5. Remove Whitespaces

In [60]:
def remove_whitespaces(text):
  return str.strip(text)

In [61]:
df['review'] = df["review"].apply(remove_whitespaces)

In [62]:
df.head()

,review,sentiment
0,one reviewers mentioned watching 1 oz e...,1
1,wonderful little production filming technique...,1
2,thought wonderful way spend Tears eyes ...,1
3,basically theres family little boy jake thi...,0
4,petter matteis love Tears eyes money vi...,1


## 6. Handling ChatWords

In [25]:
chat_words = {
    "AFAIK": "As Far As I Know",
    "AFK": "Away From Keyboard",
    "ASAP": "As Soon As Possible",
    "ATK": "At The Keyboard",
    "ATM": "At The Moment",
    "A3": "Anytime, Anywhere, Anyplace",
    "BAK": "Back At Keyboard",
    "BBL": "Be Back Later",
    "BBS": "Be Back Soon",
    "BFN": "Bye For Now",
    "B4N": "Bye For Now",
    "BRB": "Be Right Back",
    "BRT": "Be Right There",
    "BTW": "By The Way",
    "B4": "Before",
    "B4N": "Bye For Now",
    "CU": "See You",
    "CUL8R": "See You Later",
    "CYA": "See You",
    "FAQ": "Frequently Asked Questions",
    "FC": "Fingers Crossed",
    "FWIW": "For What It's Worth",
    "FYI": "For Your Information",
    "GAL": "Get A Life",
    "GG": "Good Game",
    "GN": "Good Night",
    "GMTA": "Great Minds Think Alike",
    "GR8": "Great!",
    "G9": "Genius",
    "IC": "I See",
    "ICQ": "I Seek you (also a chat program)",
    "ILU": "ILU: I Love You",
    "IMHO": "In My Honest/Humble Opinion",
    "IMO": "In My Opinion",
    "IOW": "In Other Words",
    "IRL": "In Real Life",
    "KISS": "Keep It Simple, Stupid",
    "LDR": "Long Distance Relationship",
    "LMAO": "Laugh My A.. Off",
    "LOL": "Laughing Out Loud",
    "LTNS": "Long Time No See",
    "L8R": "Later",
    "MTE": "My Thoughts Exactly",
    "M8": "Mate",
    "NRN": "No Reply Necessary",
    "OIC": "Oh I See",
    "PITA": "Pain In The A..",
    "PRT": "Party",
    "PRW": "Parents Are Watching",
    "QPSA?": "Que Pasa?",
    "ROFL": "Rolling On The Floor Laughing",
    "ROFLOL": "Rolling On The Floor Laughing Out Loud",
    "ROTFLMAO": "Rolling On The Floor Laughing My A.. Off",
    "SK8": "Skate",
    "STATS": "Your sex and age",
    "ASL": "Age, Sex, Location",
    "THX": "Thank You",
    "TTFN": "Ta-Ta For Now!",
    "TTYL": "Talk To You Later",
    "U": "You",
    "U2": "You Too",
    "U4E": "Yours For Ever",
    "WB": "Welcome Back",
    "WTF": "What The F...",
    "WTG": "Way To Go!",
    "WUF": "Where Are You From?",
    "W8": "Wait...",
    "7K": "Sick:-D Laugher",
    "TFW": "That feeling when",
    "MFW": "My face when",
    "MRW": "My reaction when",
    "IFYP": "I feel your pain",
    "TNTL": "Trying not to laugh",
    "JK": "Just kidding",
    "IDC": "I don't care",
    "ILY": "I love you",
    "IMU": "I miss you",
    "ADIH": "Another day in hell",
    "ZZZ": "Sleeping, bored, tired",
    "WYWH": "Wish you were here",
    "TIME": "Tears in my eyes",
    "BAE": "Before anyone else",
    "FIMH": "Forever in my heart",
    "BSAAW": "Big smile and a wink",
    "BWL": "Bursting with laughter",
    "BFF": "Best friends forever",
    "CSL": "Can't stop laughing"
}

In [26]:
from types import new_class
def chat_conversation(text):
  new_text = []
  for i in text.split():
    if i.upper() in chat_words:
      new_text.append(chat_words[i.upper()])
    else:
        new_text.append(i)
  return ' '.join(new_text)

In [27]:
# Eg.
text = 'FYI Delhi is the capital-of India'
chat_conversation(text)

'For Your Information Delhi is the capital-of India'

In [28]:
# We can correct spellings from whole corpous
df['review'] = df['review'].apply(chat_conversation)
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production the filming tech...,1
2,i thought this was a wonderful way to spend Te...,1
3,basically theres a family where a little boy j...,0
4,petter matteis love in the Tears in my eyes of...,1


## 6. Spelling Correction

In [29]:
from textblob import TextBlob

In [30]:
# Eg.
text = 'The cat sat on the cuchion. while plyaiing'
textBlb = TextBlob(text)
print(textBlb)

The cat sat on the cuchion. while plyaiing


In [31]:
textBlb.correct().string

'The cat sat on the cushion. while playing'

In [32]:
def correct_spellings(text):
  return str(TextBlob(text).correct())

In [33]:
# # We can correct spellings from whole corpous. use if ony have GPU
# df['review'] = df['review'].apply(correct_spellings)
# df.head()

## 7. Handling StopWords

In [34]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [35]:
from nltk.corpus import stopwords

In [36]:
stopwords = stopwords.words('english')

In [37]:
def remove_stopwords(text):
    new_text = []

    for word in text.split():
        if word in stopwords:
            new_text.append('')

        else:
            new_text.append(word)

    x = new_text[:]

    new_text.clear()

    return " ".join(x)

In [38]:
# Eg.
text = 'probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it\'s not preachy or boring. it just never gets old, despite my having seen it some 15 or more times'

remove_stopwords(text)

'probably  all-time favorite movie,  story  selflessness, sacrifice  dedication   noble cause,    preachy  boring.   never gets old, despite   seen   15   times'

In [39]:
df['review'] = df['review'].apply(remove_stopwords)

In [40]:
df.head()

,review,sentiment
0,one reviewers mentioned watching 1 oz e...,1
1,wonderful little production filming techniqu...,1
2,thought wonderful way spend Tears eyes ...,1
3,basically theres family little boy jake thi...,0
4,petter matteis love Tears eyes money vi...,1


## 8. Handeling Emojis

In [41]:
# Function to remove emojis
def remove_emojis(text):
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002500-\U00002BEF"  # chinese char
        "\U00002702-\U000027B0"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "\U0001f926-\U0001f937"
        "\U00010000-\U0010ffff"
        "\u2640-\u2642"
        "\u2600-\u2B55"
        "\u200d"
        "\u23cf"
        "\u23e9"
        "\u231a"
        "\u3030"
        "\ufe0f"
        "\u2069"
        "\u2066"
        "\u200c"
        "\u2068"
        "\u2067"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Apply the function to the DataFrame
df['review'] = df['review'].apply(remove_emojis)

In [42]:
# # Simply Convert Emojis into text it is depend only on the dataset
# !pip install emoji
# import emoji

# def enoji_to_text(text):
#   return emoji.demojize(text)

# df['review'] = df['review'].apply(enoji_to_text)

In [43]:
df.tail()

,review,sentiment
49995,thought movie right good job wasnt crea...,1
49996,bad plot bad dialogue bad acting idiotic direc...,0
49997,catholic taught parochial elementary schoo...,0
49998,im going disagree previous comment side ...,0
49999,one expects star trek movies high art fa...,0


##9. Normalize Contractions

In [64]:
#!pip install contractions
import contractions

In [65]:
def expand_contraction(text):
  expanded_words = []
  for word in text.split():
    expanded_words.append(contractions.fix(word))
  return ' '.join(expanded_words)

In [66]:
# Eg.
text = '''I'll be there within 5 min. Shouldn't you be there too?
          I'd love to see u there my dear. It's awesome to meet new friends.
          We've been waiting for this day for so long.'''

expand_contraction(text)

'I will be there within 5 min. Should not you be there too? I would love to see you there my dear. It is awesome to meet new friends. We have been waiting for this day for so long.'

In [67]:
df['review'] = df['review'].apply(expand_contraction)

In [69]:
df.tail()

,review,sentiment
49995,thought movie right good job was not creative ...,1
49996,bad plot bad dialogue bad acting idiotic direc...,0
49997,catholic taught parochial elementary schools n...,0
49998,i am going disagree previous comment side malt...,0
49999,one expects star trek movies high art fans exp...,0


##10. Tokenization

Here we can use NLTK or spaCy.
There are substantial differences between NLTK and spaCy, which are as follows:

* NLTK provides a plethora of algorithms to choose from for a particular problem which is boon for a researcher but a bane for a developer. Whereas, spaCy keeps the best algorithm for a problem in its toolkit and keep it updated as state of the art improves.

* NLTK supports various languages whereas spaCy have statistical models for 7 languages (English, German, Spanish, French, Portuguese, Italian, and Dutch). It also supports named entities for multi language.

* NLTK is a string processing library. It takes strings as input and returns strings or lists of strings as output. Whereas, spaCy uses object-oriented approach. When we parse a text, spaCy returns document object whose words and sentences are objects themselves.

* **spaCy has support for word vectors whereas NLTK does not.**

* As spaCy uses the latest and best algorithms, its performance is usually good as compared to NLTK. In word tokenization and POS-tagging spaCy performs better, but in sentence tokenization, NLTK outperforms spaCy. Its poor performance in sentence tokenization is a result of differing approaches: NLTK attempts to split the text into sentences. In contrast, spaCy constructs a syntactic tree for each sentence, a more robust method that yields much more information about the text.



In [44]:
# Here we need word tokanization so we use spaCy
import spacy
nlp = spacy.load("en_core_web_sm")
import en_core_web_sm
nlp = en_core_web_sm.load()

In [45]:
print('Eg. 1')
doc1 = nlp("This is a sentence.")
print([(w.text, w.pos_) for w in doc1])

print('Eg. 2')
doc2 =nlp("We're here to help! mail us at nks@gmail.com")
print([(w.text, w.pos_) for w in doc2])

Eg. 1
[('This', 'PRON'), ('is', 'AUX'), ('a', 'DET'), ('sentence', 'NOUN'), ('.', 'PUNCT')]
Eg. 2
[('We', 'PRON'), ("'re", 'AUX'), ('here', 'ADV'), ('to', 'PART'), ('help', 'VERB'), ('!', 'PUNCT'), ('mail', 'VERB'), ('us', 'PRON'), ('at', 'ADP'), ('nks@gmail.com', 'PROPN')]


In [46]:
def spacy_tokanizer(text):
  doc = nlp(text)
  # Return a list of token texts
  return [token.text for token in doc]

In [47]:
# E.g.
A = "This is a sentence."
spacy_tokanizer(A)

['This', 'is', 'a', 'sentence', '.']

OP2 Using NLTK  for tokanization

In [48]:
# # We can use NLTK also
# import nltk
# nltk.download('punkt')

# from nltk import word_tokenize, sent_tokenize

# #Eg.
# # Text
# sentence = 'Lorem Ipsum is simply dummy text of the printing and typesetting industry?'
# # Calling tool
# word_tokenize(sentence)

## 10. Stemming

In [49]:
from nltk.stem.porter import PorterStemmer

In [50]:
stemmer = PorterStemmer()

def stem_words(text):
  return " ".join([stemmer.stem(word) for word in text.split()])

In [51]:
# Eg.
text = 'If you see errors or degraded performance, download a newer compatible model'

stem_words(text)

'if you see error or degrad performance, download a newer compat model'

## Stemming Vs Lemmatization
Stemming and lemmatization are both techniques used in natural language processing (NLP) to reduce words to their base or root forms. While they aim to achieve similar goals, they differ in their approach and the level of reduction they perform.

### Stemming

Stemming is the process of reducing words to their root forms, which may not always be valid words. It involves cutting off prefixes or suffixes to produce the root form, known as the stem. Stemming uses heuristic algorithms that are simpler and faster compared to lemmatization.

**Key characteristics of stemming:**

- **Rule-based**: Stemming algorithms apply predefined rules to strip affixes from words.
- **Faster**: Because stemming relies on simple rules and does not require a dictionary lookup, it tends to be faster than lemmatization.
- **Base form**: Stemming may not always result in a valid word. For example, "running" may be stemmed to "run", but "wolves" might be stemmed to "wolv", which is not a real word.

### Lemmatization

Lemmatization, on the other hand, involves accurately reducing words to their base or dictionary form (lemma) based on their intended meaning. It takes into account the context and part of speech (POS) of a word to apply morphological analysis.

**Key characteristics of lemmatization:**

- **Context-aware**: Lemmatization considers the meaning of words and uses detailed dictionaries and rules to get to the root form.
- **Slower**: Lemmatization often involves dictionary lookups and more complex rules to ensure accuracy, which can make it slower than stemming.
- **Valid words**: Lemmatization results in valid words, ensuring that the transformed words belong to the language and have meaning.

### Comparison

- **Precision**: Lemmatization is more precise because it considers the context and part of speech of words. It produces valid words but is slower due to its complexity.
  
- **Speed**: Stemming is faster as it applies simpler rules without considering the context, but it may produce stems that are not actual words.

- **Use Cases**: Stemming is useful in applications where speed is crucial and getting a rough base form of words suffices, such as in information retrieval systems. Lemmatization is preferred in tasks requiring more accuracy and where the transformed words need to be meaningful, such as in search engines, chatbots, or sentiment analysis.

In essence, while both stemming and lemmatization aim to reduce words to their base forms, lemmatization achieves this with more accuracy and context-sensitivity, albeit at the cost of increased computational complexity and processing time compared to stemming.

## 11. Lemmatization

In [52]:
# spacy is more faster than NLTK and NLTK need more setup and downloading compared to spacy. So here we use spacy
# load the spacy model

nlp = spacy.load('en_core_web_sm')

# Lemmatize a sentence
sentence = "The striped bats are hanging on their feet for best"

In [53]:
doc = nlp(sentence)
lemmatized_sentence = [token.lemma_ for token in doc]

print(" ".join(lemmatized_sentence))


the stripe bat be hang on their foot for good
